In [5]:
from dotenv import load_dotenv
import os
from qiskit import *
from qiskit_ibm_provider import IBMProvider
from math import *
import qiskit
from qiskit.circuit.library.standard_gates import *
load_dotenv()
IBM_KEY = os.getenv("API_KEY")
provider = IBMProvider()

## Boolean Function Quantum Implementation
Boolean functions aren't reversible but oracle functions are. Oracle Function can be defined for $F(x)$ as:
$$U_F(x_1, x_2, \cdots, x_n, b) = (x_1, x_2, \cdots, x_n, b \oplus F(x))$$
This oracle function is reversible and thus quantum gates which support reversible operations can be used to implement this.

In [6]:
def oracle():
    circ = QuantumCircuit(3)
    circ.ccx(0, 1, 2)
    return circ

## Deutsch-Jozsa Algorithm Implementation

Following is the implementation of the Deutsch-Jozsa Algorithm for $f(x) = x_2 \oplus x_3 \oplus x_4$

In [7]:
# Building an oracle function for f(x)
orcl = QuantumCircuit(5)
orcl.cx(1, 4)
orcl.cx(2, 4)
orcl.cx(3, 4)
oracle = orcl.to_instruction()
orcl.draw()

q_0: ───────────────
                    
q_1: ──■────────────
       │            
q_2: ──┼────■───────
       │    │       
q_3: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_4: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘

In [8]:
# Quantum register, Classical register, Quantum Circuit creation
q = QuantumRegister(5)
c = ClassicalRegister(4)
qc = QuantumCircuit(q, c)

In [9]:
qc.x(4)
qc.h(range(5))
# Append the circuit of the oracle on the qubit register q to qc
qc.append(oracle, q)
qc.h(range(5))
for i in range(4):
    qc.measure(q[i], c[3 - i])

backend = Aer.get_backend('qasm_simulator')
qjob = execute(qc, backend, shots=1)
counts = qjob.result().get_counts()
print(counts)
# Output is not the state |0000>, implying f(x) is balanced

{'0111': 1}
